In [ ]:
from py123d.dataset.scene.scene_builder import ArrowSceneBuilder
from py123d.dataset.scene.scene_filter import SceneFilter

from py123d.common.multithreading.worker_sequential import Sequential
from py123d.common.datatypes.sensor.camera import CameraType

In [ ]:
from py123d.geometry import Point2D
import numpy as np

import torch

from py123d.geometry.polyline import Polyline2D

In [ ]:
# split = "nuplan_private_test"
# log_names = ["2021.09.29.17.35.58_veh-44_00066_00432"]


# splits = ["wopd_train"]
# splits = ["carla"]
splits = ["nuplan_private_test"]
# splits = ["av2-sensor-mini_train"]
# log_names = None


log_names = None
scene_uuids = None

scene_filter = SceneFilter(
    split_names=splits,
    log_names=log_names,
    scene_uuids=scene_uuids,
    duration_s=20,
    history_s=0.0,
    timestamp_threshold_s=20,
    shuffle=True,
    camera_types=[CameraType.CAM_F0],
)
scene_builder = ArrowSceneBuilder("/home/daniel/py123d_workspace/data")
worker = Sequential()
# worker = RayDistributed()
scenes = scene_builder.get_scenes(scene_filter, worker)

print(f"Found {len(scenes)} scenes")

In [ ]:
from typing import List, Optional, Tuple
import matplotlib.pyplot as plt
import numpy as np
from py123d.common.visualization.matplotlib.camera import add_camera_ax
from py123d.geometry import Point2D
from py123d.common.visualization.color.color import BLACK, DARK_GREY, DARKER_GREY, LIGHT_GREY, NEW_TAB_10, TAB_10
from py123d.common.visualization.color.config import PlotConfig
from py123d.common.visualization.color.default import CENTERLINE_CONFIG, MAP_SURFACE_CONFIG, ROUTE_CONFIG
from py123d.common.visualization.matplotlib.observation import (
    add_box_detections_to_ax,
    add_default_map_on_ax,
    add_ego_vehicle_to_ax,
    add_traffic_lights_to_ax,
)
from py123d.common.visualization.matplotlib.utils import add_shapely_linestring_to_ax, add_shapely_polygon_to_ax
from py123d.dataset.maps.abstract_map import AbstractMap
from py123d.dataset.maps.abstract_map_objects import AbstractLane, AbstractLaneGroup
from py123d.dataset.maps.gpkg.gpkg_map_objects import GPKGIntersection
from py123d.dataset.maps.map_datatypes import MapLayer
from py123d.dataset.scene.abstract_scene import AbstractScene


import shapely.geometry as geom


def _plot_scene_on_ax(ax: plt.Axes, scene: AbstractScene, iteration: int = 0, radius: float = 80) -> plt.Axes:

    ego_vehicle_state = scene.get_ego_state_at_iteration(iteration)
    box_detections = scene.get_box_detections_at_iteration(iteration)

    point_2d = ego_vehicle_state.bounding_box.center.state_se2.point_2d
    # add_debug_map_on_ax(ax, scene.get_map_api(), point_2d, radius=radius, route_lane_group_ids=None)
    add_default_map_on_ax(ax, scene.get_map_api(), point_2d, radius=radius, route_lane_group_ids=None)
    # add_traffic_lights_to_ax(ax, traffic_light_detections, scene.get_map_api())

    add_box_detections_to_ax(ax, box_detections)
    add_ego_vehicle_to_ax(ax, ego_vehicle_state)

    zoom = 1.0
    ax.set_xlim(point_2d.x - radius * zoom, point_2d.x + radius * zoom)
    ax.set_ylim(point_2d.y - radius * zoom, point_2d.y + radius * zoom)

    ax.set_aspect("equal", adjustable="box")
    return ax


def plot_scene_at_iteration(
    scene: AbstractScene, iteration: int = 0, radius: float = 80
) -> Tuple[plt.Figure, plt.Axes]:

    fig, ax = plt.subplots(1, 2, figsize=(18, 5))

    camera = scene.get_camera_at_iteration(iteration=iteration, camera_type=CameraType.CAM_F0)
    add_camera_ax(ax[0], camera)

    _plot_scene_on_ax(ax[1], scene, iteration, radius)
    for ax_ in ax:
        ax_.set_xticks([])
        ax_.set_yticks([])
    fig.subplots_adjust(wspace=-0.5)  # Make the border between axes super small
    fig.tight_layout()
    return fig, ax


scene_index = 6

iteration = 99
fig, ax = plot_scene_at_iteration(scenes[scene_index], iteration=iteration, radius=35)
plt.show()

camera = scenes[scene_index].get_camera_at_iteration(
    iteration=iteration, camera_type=CameraType.CAM_F0
)

plt.imshow(camera.image, cmap="gray", vmin=0, vmax=255)
# # fig.savefig(f"/home/daniel/scene_{scene_index}_iteration_1.pdf", dpi=300, bbox_inches="tight")

scenes[scene_index].log_name

In [ ]:
from pathlib import Path
from typing import Union

from matplotlib import animation
from tqdm import tqdm


def render_scene_animation(
    scene: AbstractScene,
    output_path: Union[str, Path],
    start_idx: int = 0,
    end_idx: Optional[int] = None,
    step: int = 5,
    fps: float = 2.0,
    dpi: int = 300,
    format: str = "gif",
    radius: float = 35,
) -> None:
    assert format in ["mp4", "gif"], "Format must be either 'mp4' or 'gif'."
    output_path = Path(output_path)
    output_path.mkdir(parents=True, exist_ok=True)

    scene.open()

    if end_idx is None:
        end_idx = scene.number_of_iterations
    end_idx = min(end_idx, scene.number_of_iterations)
    fig, ax = plt.subplots(1, 2, figsize=(18, 5))
    gs = fig.add_gridspec(1, 2, width_ratios=[6, 1])
    ax[0].set_position(gs[0].get_position(fig))
    ax[1].set_position(gs[1].get_position(fig))

    def update(i):
        ax[0].clear()
        ax[1].clear()
        for ax_ in ax:
            ax_.set_xticks([])
            ax_.set_yticks([])
        _plot_scene_on_ax(ax[1], scene, i, radius)
        camera = scene.get_camera_at_iteration(iteration=i, camera_type=CameraType.CAM_F0)
        add_camera_ax(ax[0], camera)
        fig.subplots_adjust(wspace=-0.33, hspace=0.0)
        plt.subplots_adjust(left=0.01, right=0.99, top=0.99, bottom=0.01)  # Remove all margins
        pbar.update(1)

    frames = list(range(start_idx, end_idx, step))
    pbar = tqdm(total=len(frames), desc=f"Rendering {scene.log_name} as {format}")
    ani = animation.FuncAnimation(fig, update, frames=frames, repeat=False)

    ani.save(output_path / f"{scene.log_name}_{scene.token}.{format}", writer="ffmpeg", fps=fps, dpi=dpi)
    plt.close(fig)
    scene.close()


render_scene_animation(scenes[scene_index], output_path="/home/daniel/scene_renders", format="gif", dpi=100, end_idx=200)

In [ ]:
from py123d.dataset.conversion.map.road_edge.road_edge_2d_utils import get_road_edge_linear_rings
from py123d.dataset.maps.gpkg.gpkg_map import GPKGMap


map_api: GPKGMap = scenes[scene_index].map_api

drivable_polygons = map_api._gpd_dataframes[MapLayer.LANE]



linear_rings = get_road_edge_linear_rings(drivable_polygons.geometry.tolist())
rings_lengths = [ring.length for ring in linear_rings]
max_length_idx = np.argmax(rings_lengths)






size = 16
fig, ax = plt.subplots(figsize=(size, size))

for idx, ring in enumerate(linear_rings):
    if idx == max_length_idx:
        ax.plot(*ring.xy, color="black", linewidth=2, label="Longest Road Edge")
    else:
        ax.plot(*ring.xy)


ax.set_aspect("equal", adjustable="box")